In [2]:
from astropy.io import fits

In [7]:
hdus = fits.open("../tests/data/WFPC2u57.fits")

In [8]:
bytes = open("../tests/data/WFPC2u57.fits", "rb").read()

In [9]:
def get_first_header_size(bytes):
    # Header may be multiple of 2880 bytes, get the size of the header until the block where the END card is present
    header_size = 0
    while True:
        header_size += 2880
        if b"END     " in bytes[header_size:header_size + 2880]:
            header_size += 2880
            break
        
    return header_size


In [10]:
def get_data_size_in_bytes(header):
    ## It may have multiple NAXISn keywords, so we need to get the product of all of them
    size_in_bytes = 1
    for i in range(1, header["NAXIS"] + 1):
        size_in_bytes *= header["NAXIS" + str(i)]
    
    size_in_bytes *= abs(header["BITPIX"] / 8)
    ## Make it multiple of 2880 bytes
    if size_in_bytes % 2880 != 0:
        size_in_bytes += 2880 - (size_in_bytes % 2880)
    
    return int(size_in_bytes)
    

In [11]:
second_hdus_start = get_first_header_size(bytes) + get_data_size_in_bytes(hdus[0].header)
second_hdu_bytes = bytes[second_hdus_start:]

In [12]:
get_first_header_size(second_hdu_bytes) + get_data_size_in_bytes(hdus[1].header)

34560

In [15]:
data_second_hdu = second_hdu_bytes[get_first_header_size(second_hdu_bytes):]

In [13]:
hdus[1].header

XTENSION= 'TABLE   '           / Ascii table extension                          
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
NAXIS1  =                  796 / Number of characters per row                   
NAXIS2  =                    4 / The number of rows                             
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                   49 / Number of fields per row                       
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
                                                                                
TTYPE1  = 'CRVAL1  '           /                                                
TBCOL1  =                    1 /                                                
TFORM1  = 'D25.17  '        

In [125]:
class Column:
    number: int
    ttype: str
    tform: bytes
    
    def __init__(self, number, ttype, tform, tbcol):
        self.number = number
        self.ttype = ttype
        self.tform = tform
        self.tbcol = tbcol
        self.values = []
    
    def print(self):
        print(f"Column {self.number}: {self.ttype} {self.tform} {self.values}")

In [139]:
header = hdus[1].header
columns = []
for i in range(1, header["TFIELDS"] + 1):
    columns.append(Column(i, header["TTYPE" + str(i)], header["TFORM" + str(i)], header["TBCOL" + str(i)]))

In [140]:
## if tform is E15.2 I want to get the 15
def get_tform_type_size(tform):
    if len(tform.strip()) == 1:
        size = None
    else:
        size = int(tform[1:].split(".")[0])
        
    return tform[:1], size

In [143]:
pos = 0

for line in range(1, header["NAXIS2"]):
    for key, column in enumerate(columns):
        typ, size = get_tform_type_size(column.tform)
        
        if typ == "D":
            column.values.append(float(data_second_hdu[pos:pos + size]))
        elif typ == "E":
            column.values.append(float(data_second_hdu[pos:pos + size]))
        elif typ == "I":
            column.values.append(int(data_second_hdu[pos:pos + size]))
        elif typ == "A":
            column.values.append(data_second_hdu[pos:pos + size].decode("ascii"))
        else:
            column.values.append(data_second_hdu[pos:pos + size])
        
        pos += size + 1

In [144]:
columns[1].print()

Column 2: CRVAL2 D25.17 [39.39633673411, 39.39633673411, 39.41214313815, 39.410944281389995]


In [3]:
from astropy.table import Table
t = Table.read("../tests/data/WFPC2u57.fits", format="fits")
t

CRVAL1,CRVAL2,CRPIX1,CRPIX2,CD1_1,CD1_2,CD2_1,CD2_2,DATAMIN,DATAMAX,MIR_REVR,ORIENTAT,FILLCNT,ERRCNT,FPKTTIME,LPKTTIME,CTYPE1,CTYPE2,DETECTOR,DEZERO,BIASEVEN,BIASODD,GOODMIN,GOODMAX,DATAMEAN,GPIXELS,SOFTERRS,CALIBDEF,STATICD,ATODSAT,DATALOST,BADPIXEL,OVERLAP,PHOTMODE,PHOTFLAM,PHOTZPT,PHOTPLAM,PHOTBW,MEDIAN,MEDSHADO,HISTWIDE,SKEWNESS,MEANC10,MEANC25,MEANC50,MEANC100,MEANC200,MEANC300,BACKGRND
,,,,,,,,,,LOGICAL-,,,,,,CHARACTER*8,CHARACTER*8,,,,,,,,,,,,,,,,CHARACTER*48,,,,,,,,,,,,,,,
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bytes1,float64,int64,int64,float64,float64,bytes8,bytes8,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,int64,int64,int64,int64,int64,bytes48,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
182.6311886308,39.39633673411,420,424.5,-1.06704e-06,-1.25958e-05,-1.26016e-05,1.06655e-06,-73.19537,3777.701,T,-85.16,0,0,51229.79857399999,51229.79874200001,RA---TAN,DEC--TAN,1,316.6452,316.6715,316.6189,-5.064006,2552.17,0.4182382,632387,0,1466,0,16,0,0,0,"WFPC2,1,A2D7,LRF#4877.0,,CAL",3.44746e-16,-21.1,4884.258,20.20996,-0.175651,-0.121681,1.033711,-1.983727,0.12958,0.3129676,0.4577668,0.3916293,0.3115222,0.3295493,-0.3676353
182.6255233634,39.41214313815,423.5,414,-2.75605e-05,2.08221e-06,2.08021e-06,2.75871e-05,-19.95751,3740.623,T,4.31638,0,0,51229.79874200001,51229.79891000002,RA---TAN,DEC--TAN,2,354.0573,354.0761,354.0386,-3.379329,3720.573,0.6385469,631340,0,2072,0,25,0,0,0,"WFPC2,2,A2D7,LRF#4877.0,,CAL",3.382652e-16,-21.1,4884.261,20.20992,-0.1896807,-0.1899146,1.053666,-1.947547,0.2564496,0.1057321,0.9952926,0.8409855,0.6534808,0.5836062,0.2101429
182.6523792305,39.41094428138999,436,424.5,2.23913e-06,2.75426e-05,2.75684e-05,-2.23704e-06,-58.53249,3282.136,T,94.6434,0,0,51229.79891000002,51229.799078,RA---TAN,DEC--TAN,3,306.6676,306.6541,306.681,-3.713645,3282.136,0.5217843,633259,0,1551,0,7,0,0,0,"WFPC2,3,A2D7,LRF#4877.0,,CAL",3.372288e-16,-21.1,4884.255,20.21006,0.01109185,0.06184989,1.003236,28.21703,1.669304,0.3472891,0.327215,0.6145074,0.7027547,0.5681439,0.4761558
182.650022355,39.38946044168,423,421,2.75358e-05,-2.50931e-06,-2.50689e-06,-2.75624e-05,-72.41166,3420.24,T,-174.798,0,0,51229.799079,51229.79924599999,RA---TAN,DEC--TAN,4,311.7072,311.7648,311.6496,-3.289304,3420.24,0.7084697,628289,0,1816,0,57,0,0,0,"WFPC2,4,A2D7,LRF#4877.0,,CAL",3.46466e-16,-21.1,4884.256,20.20999,-0.06813275,-0.03253493,0.9846212,-4.591252,0.1359666,0.04015671,0.1129934,1.897491,0.9687142,1.05377,0.3466465


In [154]:
data_second_hdu[800: ]

b' 1.8262552336340000E2      3.9412143138150002E1      4.235000E2      4.140000E2    -2.756050E-5     2.082210E-6     2.080210E-6     2.758710E-5     -1.995751E1      3.740623E3 T      4.316380E0            0            0      5.1229798742000003E4      5.1229798910000017E4 RA---TAN DEC--TAN            2      3.540573E2      3.540761E2      3.540386E2     -3.379329E0      3.720573E3     6.385469E-1       631340            0         2072            0           25            0            0            0 WFPC2,2,A2D7,LRF#4877.0,,CAL                        3.382652E-16     -2.110000E1      4.884261E3      2.020992E1    -1.896807E-1    -1.899146E-1      1.053666E0     -1.947547E0     2.564496E-1     1.057321E-1     9.952926E-1     8.409855E-1     6.534808E-1     5.836062E-1     2.101429E-1      1.8265237923050001E2      3.9410944281389994E1      4.360000E2      4.245000E2     2.239130E-6     2.754260E-5     2.756840E-5    -2.237040E-6     -5.853249E1      3.282136E3 T      9.464340E1         

In [155]:
hdus[1].header

XTENSION= 'TABLE   '           / Ascii table extension                          
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
NAXIS1  =                  796 / Number of characters per row                   
NAXIS2  =                    4 / The number of rows                             
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                   49 / Number of fields per row                       
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
                                                                                
TTYPE1  = 'CRVAL1  '           /                                                
TBCOL1  =                    1 /                                                
TFORM1  = 'D25.17  '        

In [45]:
Table.read("../tests/data/outputs/test_table.fits")

ValueError: could not convert string to float: b'0.367635            182.6'; the header may be missing the necessary TNULL1 keyword or the table contains invalid data

In [16]:
data_second_hdu

b'     1.8263118863080002E2      3.9396336734110001E1      4.200000E2      4.245000E2    -1.067040E-6    -1.259580E-5    -1.260160E-5     1.066550E-6     -7.319537E1      3.777701E3 T     -8.516000E1            0            0      5.1229798573999989E4      5.1229798742000003E4 RA---TAN DEC--TAN            1      3.166452E2      3.166715E2      3.166189E2     -5.064006E0      2.552170E3     4.182382E-1       632387            0         1466            0           16            0            0            0 WFPC2,1,A2D7,LRF#4877.0,,CAL                        3.447460E-16     -2.110000E1      4.884258E3      2.020996E1    -1.756510E-1    -1.216810E-1      1.033711E0     -1.983727E0     1.295800E-1     3.129676E-1     4.577668E-1     3.916293E-1     3.115222E-1     3.295493E-1    -3.676353E-1      1.8262552336340000E2      3.9412143138150002E1      4.235000E2      4.140000E2    -2.756050E-5     2.082210E-6     2.080210E-6     2.758710E-5     -1.995751E1      3.740623E3 T      4.316380E0     

In [25]:
len("     1.8263118863080002E2      3.9396336734110001E1 ")

52

In [26]:
26 + 26

52

In [27]:
he = """XTENSION= 'TABLE   ' / Ascii table extension                                    BITPIX  =                    8 / 8-bits per 'pixels'                            NAXIS   =                    2 / Simple 2-D matrix                              PCOUNT  =                    0 / No 'random' parameters                         GCOUNT  =                    1 / Only one group                                 EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               CRVAL1  = 'right ascension of reference pixel (deg)' /                          CRVAL2  = 'declination of reference pixel (deg)' /                              CRPIX1  = 'x-coordinate of reference pixel' /                                   CRPIX2  = 'y-coordinate of reference pixel' /                                   CD1_1   = 'partial of the right ascension w.r.t. x' /                           CD1_2   = 'partial of the right ascension w.r.t. y' /                           CD2_1   = 'partial of the declination w.r.t. x' /                               CD2_2   = 'partial of the declination w.r.t. y' /                               DATAMIN = 'minimum value of the data' /                                         DATAMAX = 'maximum value of the data' /                                         MIR_REVR= 'is the image mirror reversed?' /                                     ORIENTAT= 'orientation of the image (deg)' /                                    FILLCNT = 'number of segments containing fill' /                                ERRCNT  = 'number of segments containing errors' /                              FPKTTIME= 'time of the first packet (Modified Julian Date)' /                   LPKTTIME= 'time of the last packet (Modified Julian Date)' /                    CTYPE1  = 'first coordinate type' /                                             CTYPE2  = 'second coordinate type' /                                            DETECTOR= 'CCD detector: PC 1, WFC 2-4' /                                       DEZERO  = 'mean bias level from EED extended register' /                        BIASEVEN= 'bias level for even-numbered columns' /                              BIASODD = 'bias level for odd-numbered columns' /                               GOODMIN = 'minimum value of the "good" pixels' /                                GOODMAX = 'maximum value of the "good" pixels' /                                DATAMEAN= 'mean value of the "good" pixels' /                                   GPIXELS = 'number of "good" pixels (DQF = 0)' /                                 SOFTERRS= 'number of "soft error" pixels (DQF = 1)' /                           CALIBDEF= 'number of "calibration defect" pixels (DQF = 2)' /                   STATICD = 'number of "static defect" pixels (DQF = 4)' /                        ATODSAT = 'number of "AtoD saturated" pixels (DQF = 8)' /                       DATALOST= 'number of "data lost" pixels (DQF = 16)' /                           BADPIXEL= 'number of "generic bad" pixels (DQF = 32)' /                         OVERLAP = 'number of "image overlap" pixels (DQF = 64)' /                       PHOTMODE= 'Photometry mode' /                                                   PHOTFLAM= 'Inverse Sensitivity' /                                               PHOTZPT = 'Zero point' /                                                        PHOTPLAM= 'Pivot wavelength' /                                                  PHOTBW  = 'RMS bandwidth of the filter' /                                       MEDIAN  = 'middle data value when good qual pixels sorted' /                    MEDSHADO= 'median pixel value in shadow of pyramid edge' /                      HISTWIDE= 'width of the histogram' /                                            SKEWNESS= 'skewness of the histogram' /                                         MEANC10 = 'mean of a 10x10 region at center of chip' /                          MEANC25 = 'mean of a 25x25 region at center of chip' /                          MEANC50 = 'mean of a 50x50 region at center of chip' /                          MEANC100= 'mean of a 100x100 region at center of chip' /                        MEANC200= 'mean of a 200x200 region at center of chip' /                        MEANC300= 'mean of a 300x300 region at center of chip' /                        BACKGRND= 'estimated background level' /                                        TFIELDS =                   49 / Number of fields per row                       NAXIS1  =                  727 / Number of bytes in row                         NAXIS2  =                    4 / Number of rows                                 TTYPE1  = 'CRVAL1  ' / Name of field                                            TFORM1  = 'D25.17' / Format of field                                            TBCOL1  =                    1 / Starting byte of field                         TTYPE2  = 'CRVAL2  ' / Name of field                                            TFORM2  = 'D25.17' / Format of field                                            TBCOL2  =                   27 / Starting byte of field                         TTYPE3  = 'CRPIX1  ' / Name of field                                            TFORM3  = 'E15.7' / Format of field                                             TBCOL3  =                   53 / Starting byte of field                         TTYPE4  = 'CRPIX2  ' / Name of field                                            TFORM4  = 'E15.7' / Format of field                                             TBCOL4  =                   69 / Starting byte of field                         TTYPE5  = 'CD1_1   ' / Name of field                                            TFORM5  = 'E15.7' / Format of field                                             TBCOL5  =                   85 / Starting byte of field                         TTYPE6  = 'CD1_2   ' / Name of field                                            TFORM6  = 'E15.7' / Format of field                                             TBCOL6  =                  101 / Starting byte of field                         TTYPE7  = 'CD2_1   ' / Name of field                                            TFORM7  = 'E15.7' / Format of field                                             TBCOL7  =                  117 / Starting byte of field                         TTYPE8  = 'CD2_2   ' / Name of field                                            TFORM8  = 'E15.7' / Format of field                                             TBCOL8  =                  133 / Starting byte of field                         TTYPE9  = 'DATAMIN ' / Name of field                                            TFORM9  = 'E15.7' / Format of field                                             TBCOL9  =                  149 / Starting byte of field                         TTYPE10 = 'DATAMAX ' / Name of field                                            TFORM10 = 'E15.7' / Format of field                                             TBCOL10 =                  165 / Starting byte of field                         TTYPE11 = 'MIR_REVR' / Name of field                                            TFORM11 = 'A1' / Format of field                                                TBCOL11 =                  181 / Starting byte of field                         TTYPE12 = 'ORIENTAT' / Name of field                                            TFORM12 = 'E15.7' / Format of field                                             TBCOL12 =                  183 / Starting byte of field                         TTYPE13 = 'FILLCNT ' / Name of field                                            TFORM13 = 'I12' / Format of field                                               TBCOL13 =                  199 / Starting byte of field                         TTYPE14 = 'ERRCNT  ' / Name of field                                            TFORM14 = 'I12' / Format of field                                               TBCOL14 =                  212 / Starting byte of field                         TTYPE15 = 'FPKTTIME' / Name of field                                            TFORM15 = 'D25.17' / Format of field                                            TBCOL15 =                  225 / Starting byte of field                         TTYPE16 = 'LPKTTIME' / Name of field                                            TFORM16 = 'D25.17' / Format of field                                            TBCOL16 =                  251 / Starting byte of field                         TTYPE17 = 'CTYPE1  ' / Name of field                                            TFORM17 = 'A8' / Format of field                                                TBCOL17 =                  277 / Starting byte of field                         TTYPE18 = 'CTYPE2  ' / Name of field                                            TFORM18 = 'A8' / Format of field                                                TBCOL18 =                  286 / Starting byte of field                         TTYPE19 = 'DETECTOR' / Name of field                                            TFORM19 = 'I12' / Format of field                                               TBCOL19 =                  295 / Starting byte of field                         TTYPE20 = 'DEZERO  ' / Name of field                                            TFORM20 = 'E15.7' / Format of field                                             TBCOL20 =                  308 / Starting byte of field                         TTYPE21 = 'BIASEVEN' / Name of field                                            TFORM21 = 'E15.7' / Format of field                                             TBCOL21 =                  324 / Starting byte of field                         TTYPE22 = 'BIASODD ' / Name of field                                            TFORM22 = 'E15.7' / Format of field                                             TBCOL22 =                  340 / Starting byte of field                         TTYPE23 = 'GOODMIN ' / Name of field                                            TFORM23 = 'E15.7' / Format of field                                             TBCOL23 =                  356 / Starting byte of field                         TTYPE24 = 'GOODMAX ' / Name of field                                            TFORM24 = 'E15.7' / Format of field                                             TBCOL24 =                  372 / Starting byte of field                         TTYPE25 = 'DATAMEAN' / Name of field                                            TFORM25 = 'E15.7' / Format of field                                             TBCOL25 =                  388 / Starting byte of field                         TTYPE26 = 'GPIXELS ' / Name of field                                            TFORM26 = 'I12' / Format of field                                               TBCOL26 =                  404 / Starting byte of field                         TTYPE27 = 'SOFTERRS' / Name of field                                            TFORM27 = 'I12' / Format of field                                               TBCOL27 =                  417 / Starting byte of field                         TTYPE28 = 'CALIBDEF' / Name of field                                            TFORM28 = 'I12' / Format of field                                               TBCOL28 =                  430 / Starting byte of field                         TTYPE29 = 'STATICD ' / Name of field                                            TFORM29 = 'I12' / Format of field                                               TBCOL29 =                  443 / Starting byte of field                         TTYPE30 = 'ATODSAT ' / Name of field                                            TFORM30 = 'I12' / Format of field                                               TBCOL30 =                  456 / Starting byte of field                         TTYPE31 = 'DATALOST' / Name of field                                            TFORM31 = 'I12' / Format of field                                               TBCOL31 =                  469 / Starting byte of field                         TTYPE32 = 'BADPIXEL' / Name of field                                            TFORM32 = 'I12' / Format of field                                               TBCOL32 =                  482 / Starting byte of field                         TTYPE33 = 'OVERLAP ' / Name of field                                            TFORM33 = 'I12' / Format of field                                               TBCOL33 =                  495 / Starting byte of field                         TTYPE34 = 'PHOTMODE' / Name of field                                            TFORM34 = 'A28' / Format of field                                               TBCOL34 =                  508 / Starting byte of field                         TTYPE35 = 'PHOTFLAM' / Name of field                                            TFORM35 = 'E15.7' / Format of field                                             TBCOL35 =                  537 / Starting byte of field                         TTYPE36 = 'PHOTZPT ' / Name of field                                            TFORM36 = 'E15.7' / Format of field                                             TBCOL36 =                  553 / Starting byte of field                         TTYPE37 = 'PHOTPLAM' / Name of field                                            TFORM37 = 'E15.7' / Format of field                                             TBCOL37 =                  569 / Starting byte of field                         TTYPE38 = 'PHOTBW  ' / Name of field                                            TFORM38 = 'E15.7' / Format of field                                             TBCOL38 =                  585 / Starting byte of field                         TTYPE39 = 'MEDIAN  ' / Name of field                                            TFORM39 = 'E15.7' / Format of field                                             TBCOL39 =                  601 / Starting byte of field                         TTYPE40 = 'MEDSHADO' / Name of field                                            TFORM40 = 'E15.7' / Format of field                                             TBCOL40 =                  617 / Starting byte of field                         TTYPE41 = 'HISTWIDE' / Name of field                                            TFORM41 = 'E15.7' / Format of field                                             TBCOL41 =                  633 / Starting byte of field                         TTYPE42 = 'SKEWNESS' / Name of field                                            TFORM42 = 'E15.7' / Format of field                                             TBCOL42 =                  649 / Starting byte of field                         TTYPE43 = 'MEANC10 ' / Name of field                                            TFORM43 = 'E15.7' / Format of field                                             TBCOL43 =                  665 / Starting byte of field                         TTYPE44 = 'MEANC25 ' / Name of field                                            TFORM44 = 'E15.7' / Format of field                                             TBCOL44 =                  681 / Starting byte of field                         TTYPE45 = 'MEANC50 ' / Name of field                                            TFORM45 = 'E15.7' / Format of field                                             TBCOL45 =                  697 / Starting byte of field                         TTYPE46 = 'MEANC100' / Name of field                                            TFORM46 = 'E15.7' / Format of field                                             TBCOL46 =                  713 / Starting byte of field                         TTYPE47 = 'MEANC200' / Name of field                                            TFORM47 = 'E15.7' / Format of field                                             TBCOL47 =                  729 / Starting byte of field                         TTYPE48 = 'MEANC300' / Name of field                                            TFORM48 = 'E15.7' / Format of field                                             TBCOL48 =                  745 / Starting byte of field                         TTYPE49 = 'BACKGRND' / Name of field                                            TFORM49 = 'E15.7' / Format of field                                             TBCOL49 =                  761 / Starting byte of field                         END                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        182.6311886308            39.39633673411             420           424.5  -0.00000106704   -0.0000125958   -0.0000126016   0.00000106655       -73.19537        3777.701 T          -85.16            0            0        51229.798573999986        51229.798742000006 RA---TAN DEC--TAN            1        316.6452        316.6715        316.6189       -5.064006         2552.17       0.4182382       632387            0         1466            0           16            0            0            0 WFPC2,1,A2D7,LRF#4877.0,,CAL 0.000000000000000344746           -21.1        4884.258        20.20996       -0.175651       -0.121681        1.033711       -1.983727         0.12958       0.3129676       0.4577668       0.3916293       0.3115222       0.3295493      """

In [37]:
he[2880*6:2880*6 + 1000]

'           182.6311886308            39.39633673411             420           424.5  -0.00000106704   -0.0000125958   -0.0000126016   0.00000106655       -73.19537        3777.701 T          -85.16            0            0        51229.798573999986        51229.798742000006 RA---TAN DEC--TAN            1        316.6452        316.6715        316.6189       -5.064006         2552.17       0.4182382       632387            0         1466            0           16            0            0            0 WFPC2,1,A2D7,LRF#4877.0,,CAL 0.000000000000000344746           -21.1        4884.258        20.20996       -0.175651       -0.121681        1.033711       -1.983727         0.12958       0.3129676       0.4577668       0.3916293       0.3115222       0.3295493      '

In [39]:
len("           182.6311886308            39.39633673411 ")

52

In [41]:
fits.getheader("../tests/data/outputs/test_table.fits", 1)

XTENSION= 'TABLE   ' / Ascii table extension                                    
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
CRVAL1  = 'right ascension of reference pixel (deg)' /                          
CRVAL2  = 'declination of reference pixel (deg)' /                              
CRPIX1  = 'x-coordinate of reference pixel' /                                   
CRPIX2  = 'y-coordinate of reference pixel' /                                   
CD1_1   = 'partial of the right ascension w.r.t. x' /                           
CD1_2   = 'partial of the right ascension w.r.t. y' /                           
CD2_1   = 'partial of the de